# Model Development and Analysis

## Importing Libraries and Loading Data

In [2]:
import pandas as pd


In [38]:
# 1. Load the policies.csv and claims.csv dataset. 
# Loading data
policies = pd.read_csv('policies.csv')
claims = pd.read_csv('claims.csv')

In [39]:
# Checking dfs
policies.head(4)

,Policy_ID,Customer_ID,Policy_Age,Product_Type,Vehicle_Type,Driving_History,Region,Exposure,Premium,Earned_Premium,Ultimate_Losses,Pure_Premium,Reserved_Losses
0,1,861,1,Liability,Truck,1,West,2,931.31,931.31,161.58,80.790988,129.26
1,2,3773,25,Liability,SUV,1,West,26,983.91,983.91,2446.77,94.106621,1957.42
2,3,3093,20,Home,Car,2,Northeast,21,772.97,772.97,3361.05,160.049829,2688.84
3,4,467,17,BOP,Truck,0,South,18,846.10,846.10,835.60,46.422274,668.48


In [40]:
# Checking dfs
claims.head(4)

,Claim_ID,Policy_ID,Claim_Date,Claim_Amount,Claim_Type,Severity
0,1,1,2024-11-19,132.88,Property Damage,132.88
1,2,1,2024-11-12,85.09,Bodily Injury,85.09
2,3,7,2019-04-24,93.94,Property Damage,93.94
3,4,13,2013-08-18,0.00,Property Damage,0.00


## Data Cleaning and Summarization

In [41]:
policies.describe()

,Policy_ID,Customer_ID,Policy_Age,Driving_History,Exposure,Premium,Earned_Premium,Ultimate_Losses,Pure_Premium,Reserved_Losses
count,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,9950.000000
mean,5001.190149,2522.175622,15.146468,0.500995,16.146468,753.924627,753.924627,982.191186,61.657739,784.461113
std,2887.968529,1438.197320,8.841954,0.718415,8.841954,147.556858,147.556858,1430.443634,73.306950,1141.656444
min,1.000000,2.000000,0.000000,0.000000,1.000000,323.340000,323.340000,0.000000,0.000000,0.000000
25%,2499.250000,1263.250000,7.000000,0.000000,8.000000,649.257500,649.257500,0.000000,0.000000,0.000000
50%,5005.500000,2549.500000,15.000000,0.000000,16.000000,743.265000,743.265000,380.975000,36.776369,304.985000
75%,7502.750000,3762.750000,23.000000,1.000000,24.000000,847.637500,847.637500,1411.735000,99.056404,1129.385000
max,10000.000000,4999.000000,30.000000,5.000000,31.000000,1398.810000,1398.810000,12823.260000,502.411693,10258.610000


In [43]:
# 2. Identify and handle missing values in the Reserved_Losses column.
policies.isnull().sum()



Policy_ID            0
Customer_ID          0
Policy_Age           0
Product_Type         0
Vehicle_Type         0
Driving_History      0
Region               0
Exposure             0
Premium              0
Earned_Premium       0
Ultimate_Losses      0
Pure_Premium         0
Reserved_Losses    100
dtype: int64

In [ ]:
# Option 1: Imput NaN --> Imput Median proceed with this df
reserved_losses_median = policies['Reserved_Losses'].median()
policies_df = policies
policies_df['Reserved_Losses'].fillna(reserved_losses_median, inplace=False)
policies_df['Reserved_Losses'].isnull().sum()


100

In [45]:
# Option 2: Drop NaN
policies_drop_nan = policies.dropna()
pecent_dropped = round(1 - len(policies_drop_nan) / len(policies), 2) * 100
print(f'Percent Dropped: {pecent_dropped} %')


Percent Dropped: 1.0 %


In [42]:
policies.columns

Index(['Policy_ID', 'Customer_ID', 'Policy_Age', 'Product_Type',
       'Vehicle_Type', 'Driving_History', 'Region', 'Exposure', 'Premium',
       'Earned_Premium', 'Ultimate_Losses', 'Pure_Premium', 'Reserved_Losses'],
      dtype='object')

In [ ]:
# 3. Summarize the data by calculating:
#   - Average premium
#   - Total ultimate losses
#   - Average pure premium for each Product_Type
summary = policies_df.groupby('Product_Type').agg(
    Average_Premium=pd.NamedAgg(column='Premium', aggfunc='mean'),
    Total_Ultimate_Losses=pd.NamedAgg(column='Ultimate_Losses', aggfunc='sum'),
    Average_Pure_Premium=pd.NamedAgg(column='Pure_Premium', aggfunc='mean')
).reset_index()

# Checking summary
summary


,Product_Type,Average_Premium,Total_Ultimate_Losses,Average_Pure_Premium
0,Auto,718.072534,1489783.10,61.699596
1,BOP,717.749821,2012607.09,63.048020
2,Home,710.571541,2383081.43,60.939099
3,Liability,863.237989,2532333.90,61.574099
4,Transportation,721.192510,1453215.90,61.066003
